# Security: remove hard-coded credentials

I removed hard-coded Snowflake credentials from this notebook and replaced them with a secure loader that reads credentials from environment variables or prompts at runtime.

Please store credentials in a local `.env` file (do NOT commit it). Example keys (also in `.env.example`):

- SNOWFLAKE_USER
- SNOWFLAKE_PASSWORD
- SNOWFLAKE_ACCOUNT
- SNOWFLAKE_WAREHOUSE
- SNOWFLAKE_DATABASE
- SNOWFLAKE_SCHEMA
- SNOWFLAKE_ROLE

`.gitignore` already contains `.env` to prevent accidental commits. To load variables, copy `.env.example` -> `.env` and edit, or export environment variables in your shell.

The next code cell uses `python-dotenv` and `getpass` to safely obtain credentials.

In [1]:
import snowflake.connector
import pandas as pd
import dotenv
import os

In [32]:
#pip install dotenv

In [2]:
# Secure Snowflake connection loader
from dotenv import load_dotenv
import os
import getpass
import snowflake.connector
import pandas as pd

# Load .env if present
load_dotenv()

SF_USER = os.getenv("SNOWFLAKE_USER")
SF_PASSWORD = os.getenv("SNOWFLAKE_PASSWORD")
SF_ACCOUNT = os.getenv("SNOWFLAKE_ACCOUNT")
SF_WAREHOUSE = os.getenv("SNOWFLAKE_WAREHOUSE")
SF_DATABASE = os.getenv("SNOWFLAKE_DATABASE")
SF_SCHEMA = os.getenv("SNOWFLAKE_SCHEMA")
SF_ROLE = os.getenv("SNOWFLAKE_ROLE")

# Prompt for any missing secrets interactively (won't be saved in notebook)
if not SF_USER:
    SF_USER = input("Snowflake user: ")
if not SF_PASSWORD:
    SF_PASSWORD = getpass.getpass("Snowflake password (input hidden): ")

# Minimal validation
missing = [k for k, v in {
    'SNOWFLAKE_ACCOUNT': SF_ACCOUNT,
    'SNOWFLAKE_WAREHOUSE': SF_WAREHOUSE,
    'SNOWFLAKE_DATABASE': SF_DATABASE,
    'SNOWFLAKE_SCHEMA': SF_SCHEMA,
    'SNOWFLAKE_ROLE': SF_ROLE,
}.items() if not v]
if missing:
    print(f"Warning: the following env vars are not set: {missing}. You may run into errors if they are required.")

conn = snowflake.connector.connect(
    user=SF_USER,
    account=SF_ACCOUNT,
    warehouse=SF_WAREHOUSE,
    database=SF_DATABASE,
    schema=SF_SCHEMA,
    role=SF_ROLE,
    password=SF_PASSWORD
)
cur = conn.cursor()

# List all tables in the current database + schema
cur.execute("SHOW TABLES;")
tables = cur.fetchall()

for t in tables:
    print(t)

# Keep connection open for subsequent cells that call read_table
# Close when you are done: conn.close(), cur.close()


DatabaseError: 250001 (08001): Failed to connect to DB: gmrzyvu-ry13975.snowflakecomputing.com:443. Incorrect username or password was specified.

In [36]:
# Function to read a table into pandas
def read_table(table_name, conn):
    query = f"SELECT * FROM {table_name};"
    return pd.read_sql(query, conn)

# Read the tables
customer_info = read_table("CUSTOMER_INFO", conn)
fraudulent_patterns = read_table("FRAUDULENT_PATTERNS", conn)
merchant_info = read_table("MERCHANT_INFO", conn)
transactions = read_table("TRANSACTIONS", conn)
transactions_flags = read_table("TRANSACTIONS_FLAGS", conn)
transaction_patterns = read_table("TRANSACTION_PATTERNS", conn)

C:\Users\GIDI\AppData\Local\Temp\ipykernel_28716\302054816.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)
